### Todos los días del desafío:

* [Day 1: Handling missing values](https://www.kaggle.com/rtatman/data-cleaning-challenge-handling-missing-values)
* [Day 2: Scaling and normalization](https://www.kaggle.com/rtatman/data-cleaning-challenge-scale-and-normalize-data)
* [Day 3: Parsing dates](https://www.kaggle.com/rtatman/data-cleaning-challenge-parsing-dates/)
* [Day 4: Character encodings](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings/)
* [Day 5: Inconsistent Data Entry](https://www.kaggle.com/rtatman/data-cleaning-challenge-inconsistent-data-entry/)
___
¡Bienvenido al día 2 del Desafío de datos de 5 días! Hoy, veremos cómo escalar y normalizar los datos (¡y cuál es la diferencia entre los dos!). Para comenzar, haga clic en el botón azul "Fork Notebook" en la esquina superior derecha. Esto creará una copia privada de este cuaderno que puede editar y jugar. Una vez que haya terminado con los ejercicios, puede optar por hacer público su cuaderno para compartirlo con otros. :)

> **Your turn!** Mientras trabajamos en este cuaderno, verá algunas celdas del cuaderno (un bloque de código o texto) que tiene "¡Tu turno!" escrito en él. Estos son ejercicios que puede realizar para ayudar a consolidar su comprensión de los conceptos de los que estamos hablando. Una vez que haya escrito el código para responder una pregunta específica, puede ejecutar el código haciendo clic dentro de la celda (cuadro con el código) con el código que desea ejecutar y luego presione CTRL + ENTER (CMD + ENTER en una Mac ). También puede hacer clic en una celda y luego hacer clic en la flecha derecha "reproducir" a la izquierda del código. Si desea ejecutar todo el código en su cuaderno, puede usar las flechas dobles de "avance rápido" en la parte inferior del editor del cuaderno.

Esto es lo que haremos hoy:

* [Get our environment set up](#Get-our-environment-set-up)
* [Scaling vs. Normalization: What's the difference?](#Scaling-vs.-Normalization:-What's-the-difference?)
* [Practice scaling](#Practice-scaling)
* [Practice normalization](#Practice-normalization)

¡Empecemos!

# Configurar nuestro entorno
________

Lo primero que tendremos que hacer es cargar las bibliotecas y los conjuntos de datos que usaremos.

> **¡Importante!** ¡Asegúrese de ejecutar esta celda usted mismo o el resto de su código no funcionará!

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# for Box-Cox Transformation
from scipy import stats

# for min_max scaling
from mlxtend.preprocessing import minmax_scaling

# plotting modules
import seaborn as sns
import matplotlib.pyplot as plt

# read in all our data
kickstarters_2017 = pd.read_csv("../input/kickstarter-projects/ks-projects-201801.csv")

# set seed for reproducibility
np.random.seed(0)

Ahora que estamos configurados, aprendamos sobre la escala y la normalización. (Si lo desea, puede aprovechar esta oportunidad para echar un vistazo a algunos de los datos).

# Escalado frente a normalización: ¿cuál es la diferencia?
____

Una de las razones por las que es fácil confundirse entre escalamiento y normalización es porque los términos a veces se usan indistintamente y, para hacerlo aún más confuso, ¡son muy similares! En ambos casos, está transformando los valores de las variables numéricas para que los puntos de datos transformados tengan propiedades útiles específicas. La diferencia es que, al escalar, está cambiando el * rango * de sus datos mientras que en la normalización está cambiando la * forma de la distribución * de sus datos. Hablemos un poco más en profundidad sobre cada una de estas opciones. 

___

## **Scaling**

Esto significa que está transformando sus datos para que se ajusten a una escala específica, como 0-100 o 0-1. Desea escalar los datos cuando usa métodos basados en medidas de la distancia entre los puntos de datos, como [support vector machines, or SVM](https://en.wikipedia.org/wiki/Support_vector_machine) or [k-nearest neighbors, or KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm). Con estos algoritmos, un cambio de "1" en cualquier característica numérica recibe la misma importancia.

Por ejemplo, es posible que esté mirando los precios de algunos productos tanto en yenes como en dólares estadounidenses. Un dólar estadounidense vale alrededor de 100 yenes, pero si no escala sus precios, los métodos como SVM o KNN considerarán una diferencia de precio de 1 yenes tan importante como una diferencia de 1 dólar estadounidense. Esto claramente no encaja con nuestras intuiciones del mundo. Con la moneda, puede convertir entre monedas. Pero, ¿qué pasa si miras algo como la altura y el peso? No está del todo claro cuántas libras deben equivaler a una pulgada (o cuántos kilogramos deben equivaler a un metro).

Al escalar sus variables, puede ayudar a comparar diferentes variables en pie de igualdad. Para ayudar a solidificar el aspecto de la escala, veamos un ejemplo inventado. (No se preocupe, trabajaremos con datos reales en solo un segundo, esto es solo para ayudar a ilustrar mi punto).

In [ ]:
# generate 1000 data points randomly drawn from an exponential distribution
original_data = np.random.exponential(size = 1000)

# mix-max scale the data between 0 and 1
scaled_data = minmax_scaling(original_data, columns = [0])

# plot both together to compare
fig, ax=plt.subplots(1,2)
sns.distplot(original_data, ax=ax[0])
ax[0].set_title("Original Data")
sns.distplot(scaled_data, ax=ax[1])
ax[1].set_title("Scaled data")

Observe que la * forma * de los datos no cambia, pero que en lugar de variar de 0 a 8, ahora varía de 0 a 1.

___
## Normalización

El escalado solo cambia el rango de sus datos. La normalización es una transformación más radical. El punto de la normalización es cambiar sus observaciones para que puedan describirse como una distribución normal.

> **[Normal distribution:](https://en.wikipedia.org/wiki/Normal_distribution)** También conocida como "curva de campana", se trata de una distribución estadística específica en la que unas observaciones aproximadamente iguales caen por encima y por debajo de la media, la media y la mediana son iguales y hay más observaciones más cercanas a la media. La distribución normal también se conoce como distribución gaussiana.

En general, solo querrá normalizar sus datos si va a utilizar una técnica de aprendizaje automático o estadística que asume que sus datos se distribuyen normalmente. Algunos ejemplos de estos incluyen pruebas t, ANOVA, regresión lineal, análisis discriminante lineal (LDA) y Bayes ingenuo de Gauss. (Consejo profesional: cualquier método con "Gaussiano" en el nombre probablemente asume normalidad).

El método que usamos para normalizar aquí se llama [Box-Cox Transformation](https://en.wikipedia.org/wiki/Power_transform#Box%E2%80%93Cox_transformation). Echemos un vistazo rápido a cómo se ve la normalización de algunos datos:

In [ ]:
# normalize the exponential data with boxcox
normalized_data = stats.boxcox(original_data)

# plot both together to compare
fig, ax=plt.subplots(1,2)
sns.distplot(original_data, ax=ax[0])
ax[0].set_title("Original Data")
sns.distplot(normalized_data[0], ax=ax[1])
ax[1].set_title("Normalized data")

Observe que la * forma * de nuestros datos ha cambiado. Antes de normalizarse tenía casi forma de L. Pero después de normalizarlo se parece más al contorno de una campana (de ahí "curva de campana").
___
## ¡Tu turno!

Para el siguiente ejemplo, decida si la escala o la normalización tiene más sentido. 

* Desea construir un modelo de regresión lineal para predecir las calificaciones de alguien dado el tiempo que dedica a diversas actividades durante una semana escolar normal. Observa que sus mediciones de cuánto tiempo pasan estudiando los estudiantes no están distribuidas normalmente: algunos estudiantes casi no dedican tiempo a estudiar y otros estudian durante cuatro o más horas todos los días. ¿Debería escalar o normalizar esta variable?
* Todavía está trabajando en su estudio de calificaciones, pero también desea incluir información sobre el desempeño de los estudiantes en varias pruebas de aptitud física. Tiene información sobre cuántos saltos de tijera y lagartijas puede completar cada estudiante en un minuto. Sin embargo, observa que los estudiantes realizan muchos más saltos de tijera que flexiones de brazos: el promedio para los primeros es 40 y para los segundos solo 10. ¿Debería escalar o normalizar estas variables?

# Practica escalar
___

Para practicar el escalado y la normalización, utilizaremos un conjunto de datos de campañas de Kickstarter. (Kickstarter es un sitio web donde las personas pueden pedir a las personas que inviertan en varios proyectos y productos conceptuales).

Comencemos por escalar los objetivos de cada campaña, que es cuánto dinero estaban pidiendo.

In [ ]:
# select the usd_goal_real column
usd_goal = kickstarters_2017.usd_goal_real

# scale the goals from 0 to 1
scaled_data = minmax_scaling(usd_goal, columns = [0])

# plot the original & scaled data together to compare
fig, ax=plt.subplots(1,2)
sns.distplot(kickstarters_2017.usd_goal_real, ax=ax[0])
ax[0].set_title("Original Data")
sns.distplot(scaled_data, ax=ax[1])
ax[1].set_title("Scaled data")

Puede ver que la escala cambió drásticamente las escalas de los gráficos (pero no la forma de los datos: parece que la mayoría de las campañas tienen objetivos pequeños, pero algunas tienen objetivos muy grandes).

In [ ]:
# Your turn! 

# We just scaled the "usd_goal_real" column. What about the "goal" column?


# Practice normalization
___

Bien, ahora intentemos practicar la normalización. Vamos a normalizar la cantidad de dinero comprometida para cada campaña.

In [ ]:
# get the index of all positive pledges (Box-Cox only takes postive values)
index_of_positive_pledges = kickstarters_2017.usd_pledged_real > 0

# get only positive pledges (using their indexes)
positive_pledges = kickstarters_2017.usd_pledged_real.loc[index_of_positive_pledges]

# normalize the pledges (w/ Box-Cox)
normalized_pledges = stats.boxcox(positive_pledges)[0]

# plot both together to compare
fig, ax=plt.subplots(1,2)
sns.distplot(positive_pledges, ax=ax[0])
ax[0].set_title("Original Data")
sns.distplot(normalized_pledges, ax=ax[1])
ax[1].set_title("Normalized data")

No es perfecto (parece que muchas promesas obtuvieron muy pocas promesas), ¡pero está mucho más cerca de lo normal!

In [ ]:
# Your turn! 
# We looked as the usd_pledged_real column. What about the "pledged" column? Does it have the same info?


¡Y eso es todo por hoy! Si tiene alguna pregunta, asegúrese de publicarla en los comentarios a continuación o [on the forums](https://www.kaggle.com/questions-and-answers). 

Recuerde que su bloc de notas es privado de forma predeterminada y, para compartirlo con otras personas o pedir ayuda con él, deberá hacerlo público. Primero, deberá guardar una versión de su cuaderno que muestre su trabajo actual presionando el botón "Confirmar y ejecutar". (Su trabajo se guarda automáticamente, pero el control de versiones de su trabajo le permite volver atrás y ver cómo era en el momento en que lo guardó. También le permite compartir un buen cuaderno compilado en lugar de solo el código sin procesar). la computadora portátil ha terminado de ejecutarse, puede ir a la pestaña Configuración en el panel de la izquierda (es posible que deba expandirla presionando el botón [<] junto al botón "Confirmar y ejecutar") y configurar el menú desplegable "Visibilidad" en "Público".

# ¡Más práctica!
___

Intente encontrar un nuevo conjunto de datos y finja que se está preparando para realizar una [regression analysis](https://www.kaggle.com/rtatman/the-5-day-regression-challenge). ([These datasets are a good start!](https://www.kaggle.com/rtatman/datasets-for-regression-analysis)) Elija tres o cuatro variables y decida si necesita normalizar o escalar alguna de ellas y, si cree que debe hacerlo, practique la aplicación de la técnica correcta.